In [ ]:
#!/usr/bin/env python
import os
import sys
import rospy
import copy
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import moveit_msgs.msg
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from math import pi, cos, sin
import math
from numpy.linalg import inv
import scipy.io
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from poke_grasp.msg import stone_pose
from sensor_msgs.msg import Image, CameraInfo
import message_filters

import urx
import math3d as m3d
import logging

import matplotlib.pyplot as plt

rospy.init_node('pokegrasp', anonymous=True) #initialize the node
scene = moveit_commander.PlanningSceneInterface()

In [ ]:


'''
gripper_pub = rospy.Publisher('Robotiq2FGripperRobotOutput', outputMsg.Robotiq2FGripper_robot_output, queue_size=10)
servo_pub = rospy.Publisher('servo', UInt16, queue_size=10)
line_pub = rospy.Publisher('line_bbox', MarkerArray, queue_size=10)
rospy.sleep(1)
'''
logging.basicConfig(level=logging.WARN)
rob = urx.Robot("192.168.1.102")
#gripper_pub = rospy.Publisher('Robotiq2FGripperRobotOutput', outputMsg.Robotiq2FGripper_robot_output, queue_size=10)



In [ ]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()
#Robotiq.goto(robotiq_client, pos=0.008+0.006+0.008, speed=0.5, force=10, block=False)

In [ ]:
def go_to_home(a, vel):
    '''
    Hong_joint0 = math.radians(106.83)
    Hong_joint1 = math.radians(-77.46)
    Hong_joint2 = math.radians(-125.81)
    Hong_joint3 = math.radians(-68.64)
    Hong_joint4 = math.radians(89.58)
    Hong_joint5 = math.radians(-73.07)
    '''
    
    Hong_joint0 = math.radians(123.26)
    Hong_joint1 = math.radians(-91.84)
    Hong_joint2 = math.radians(-112.92)
    Hong_joint3 = math.radians(-67.13)
    Hong_joint4 = math.radians(89.86)
    Hong_joint5 = math.radians(-56.64)
    

    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), a, vel)


In [ ]:
def execute():
    global pose_x
    global pose_y
    global pose_z
    global yaw
    global pitch
    global normal
    
    ####################### change tcp pose ############################
    #rob.set_tcp((0.014/2+0.003, 0.0, 0.32601, 0, 0, 0))
    rob.set_tcp((0.01, -0.003, 0.32601, 0, 0, 0))
    #rob.set_tcp((0.013/2, 0.0, 0.28118, 0, 0, 0))
    #rob.set_tcp((0.024/2, 0, 0.36386, 0, 0, 0))
    ####################################################################
    rob.translate_tool((0, 0, -0.05-0.045), acc=0.2, vel=0.3)
    
    camPstone = np.array([pose_x, pose_y, pose_z, 1])
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173, -0.0934057, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    print eeTcam

    yaxis = (0, 1, 0)
    zaxis = (0, 0, 1)
    Ry = tf.transformations.rotation_matrix(pi/2, yaxis)
    Rz = tf.transformations.rotation_matrix(-pi/2, zaxis)
    eeTtcp = np.matmul(Ry, Rz)
    ####################### change tcp pose ############################
    #eeTtcp[:3,3] = np.array([0.32601,-0.014/2-0.003,0])
    eeTtcp[:3,3] = np.array([0.32601,-0.01,0.003])
    #eeTtcp[:3,3] = np.array([0.28118,-0.013/2,0])
    #eeTtcp[:3,3] = np.array([0.36386,-0.024/2,0])
    ####################################################################
    tcpTee = inv(eeTtcp)
    print tcpTee
    
    eeTstone = np.matmul(eeTcam.get_matrix(), camPstone)
    print eeTstone
    tcpTstone = np.matmul(tcpTee, np.transpose(eeTstone))
    print "tcpTstone", tcpTstone
    
    n_normalize = normal / np.linalg.norm(normal)
    #if n_normalize[2] > 0:
    #    n_normalize[0] = -n_normalize[0]
    #    n_normalize[1] = -n_normalize[1]
    #    n_normalize[2] = -n_normalize[2]
    print "n_normalize", n_normalize
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    eeTnormal = np.matmul(eeTcam.get_matrix(), n_normalize)
    tcpTnormal = np.matmul(tcpTee, np.transpose(eeTnormal))
    print "tcpTnormal", tcpTnormal
    alpha = math.atan2(tcpTnormal[1], tcpTnormal[0]) # is yaw
    if tcpTnormal[0] > 0:
        beta = -(pi/2+math.atan2(tcpTnormal[2], tcpTnormal[0])) #tune psi
    if tcpTnormal[0] < 0:
        beta = -(pi/2+math.atan2(tcpTnormal[2], -tcpTnormal[0])) #tune psi
#        beta = -math.atan2(tcpTnormal[2], tcpTnormal[0])-pi/2
        #beta = -(pi/2-(math.atan2(tcpTnormal[2], tcpTnormal[0])+pi))
    print "alpha", alpha*180/pi, "beta", beta*180/pi
    
    move = m3d.Transform((tcpTstone[0,0]+0.01, tcpTstone[1,0]+0.01, 0, 0, 0, 0))
    rob.add_pose_tool( move, acc=0.3, vel=0.5, wait=True, command="movel", threshold=None)
    #rob.translate_tool((-0.002*cos(beta), 0.006, tcpTstone[2,0]+0.055+0.045), acc=0.1, vel=0.2)
    rob.translate_tool((-0.008, 0.005, tcpTstone[2,0]+0.055+0.045+0.0005), acc=0.1, vel=0.2)
    
    rotation = m3d.Transform((0,0,0,0,beta,alpha))
    
    rob.add_pose_tool( rotation, acc=0.3, vel=0.5, wait=True, command="movel", threshold=None)
    #rotation = m3d.Transform((0,0,0,0,beta,0))
    #rob.add_pose_tool( rotation, acc=0.1, vel=0.1, wait=True, command="movel", threshold=None)
    
    
    #rob.translate_tool((0.003*cos(pitch), 0.008, tcpTstone[2,0]+0.2), acc=0.02, vel=0.03)
    #rob.translate_tool((0.008*cos(pitch), 0.003, tcpTstone[2,0]+0.2), acc=0.02, vel=0.03)
    
    
    #rotation = m3d.Transform((0,0,0,0,0,alpha))
    #rob.add_pose_tool( rotation, acc=0.1, vel=0.1, wait=True, command="movel", threshold=None)
    #rotation = m3d.Transform((0,0,0,0,beta,0))
    #rob.add_pose_tool( rotation, acc=0.1, vel=0.1, wait=True, command="movel", threshold=None)
    
    #rotation = m3d.Transform((0,0,0,0,math.radians(45),0))
    #rob.add_pose_tool( rotation, acc=0.05, vel=0.05, wait=True, command="movel", threshold=None)

    #rotation = m3d.Transform((0,0,0,0,0,yaw))
    #rob.add_pose_tool( rotation, acc=0.1, vel=0.1, wait=True, command="movel", threshold=None)
    
    #rob.translate_tool((0.013, -0.01, 0), acc=0.02, vel=0.03)
#    rob.translate_tool((0.006*cos(beta), 0, 0), acc=0.06, vel=0.08)
    
    ###########################Choose d and psi######################################
    rob.translate_tool((0.004+0.0015*int(abs(beta)>10)+0.003*int(abs(beta)>15), 0, 0), acc=0.06, vel=0.08)
    psi = math.radians(20)
    rotation = m3d.Transform((0,0,0,0,psi,0))
    rob.add_pose_tool( rotation, acc=0.3, vel=0.3, wait=True, command="movel", threshold=None)
    #################################################################################
    
    #rob.translate_tool((0, -0.005, 0), acc=0.02, vel=0.03)
    
    print 0.1226-(tcpTstone[2,0]+0.2)
    print "tcpTstone[2,0]= ",tcpTstone[2,0]
    
    #rob.translate_tool((0, 0, 0.1196-(tcpTstone[2,0]+0.2+0.002)), acc=0.02, vel=0.03)
    #rob.translate_tool((0, 0, 0.015-tcpTstone[2,0]-0.02-0.00-0.045-0.002), acc=0.03, vel=0.05)
    ##rob.translate_tool((0, 0, 0.0265), acc=0.03, vel=0.05)

    rob.translate_tool((0, 0, 0.0285), acc=0.01, vel=0.05) #2.85 a003 v005
    Robotiq.goto(robotiq_client, pos=0.006, speed=0.5, force=10, block=False)
    rospy.sleep(.5)
    
    #rotation = m3d.Transform((0,0,0,0,-pitch,0))
    #rob.add_pose_tool( rotation, acc=0.02, vel=0.03, wait=True, command="movel", threshold=None)
    
    #########################ONLY PICKING################################
    # return home
    #raw_input()
 #   rob.translate_tool((0, 0, -0.08), acc=0.02, vel=0.03)
 #   rob.movel((0, -0.12, 0, 0, 0, 0), acc=0.2, vel=0.2, wait=True, relative=True)
 #   go_to_home()
 #   Robotiq.goto(robotiq_client, pos=0.024, speed=0.2, force=10, block=False)
    
    
    ##########################Translating to PLACING################################
    rob.translate_tool((0, 0, -0.08), acc=0.3, vel=0.5)
    joint = rob.getj()
    if joint[5] > pi:
        rob.movej((joint[0],joint[1],joint[2],joint[3],joint[4],joint[5]-pi),0.5,0.8)
    elif joint[5] < -pi:
        rob.movej((joint[0],joint[1],joint[2],joint[3],joint[4],joint[5]+pi),0.5,0.8)
    
    rob.movel((0, -0.15, 0, 0, 0, 0), acc=0.3, vel=0.5, wait=True, relative=True)
    go_to_home(0.5, 0.7)
    Robotiq.goto(robotiq_client, pos=0.020, speed=0.5, force=10, block=False)
    
    '''
    rob.movel((-0.35, -0.25, 0, 0, 0, 0), acc=0.2, vel=0.2, wait=True, relative=True)
    rotation = m3d.Transform((0,0,0,0,0,-alpha+pi))
    rob.add_pose_tool( rotation, acc=0.1, vel=0.1, wait=True, command="movel", threshold=None)
    rob.movel((0, 0, -0.173, 0, 0, 0), acc=0.1, vel=0.1, wait=True, relative=True)
    #go_to_home()
    #Robotiq.goto(robotiq_client, pos=0.008+0.006+0.008, speed=0.2, force=10, block=False)
    '''

In [ ]:
def stone_pose_callback(msg):
    print(msg)
    
    global pose_x
    global pose_y
    global pose_z
    global yaw
    global pitch
    global normal
    
    pose_x = msg.x
    pose_y = msg.y
    pose_z = msg.z
    yaw = msg.yaw
    pitch = msg.pitch
    normal = msg.normal
    
    print 'pose_x = ', pose_x, ',pose_y = ', pose_y, ',pose_z = ', pose_z
    
    
pose_topic = '/stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(pose_topic, stone_pose, stone_pose_callback)

In [ ]:
is_send_stone_img = 0
stone_img_count = 0

pose_x = 0
pose_y = 0
yaw = 0
pitch = 0
bridge = CvBridge()

def image_callback(color, a_depth):
    global is_send_stone_img
    global stone_img_count
    
    #print 'enter img callback', is_send_img
    
    if is_send_stone_img == 1:
        print "dsag"
        #print(color)
        #print(a_depth)
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        rospy.sleep(0.5)
        cv2_img = bridge.imgmsg_to_cv2(color, "bgr8")
        cv2.imwrite('../../../../tensorflow_proj/Mask_RCNN/samples/stones/JPEGImages/'+str(stone_img_count)+'.jpeg', cv2_img)
        cv2_depth_img = bridge.imgmsg_to_cv2(a_depth, desired_encoding="passthrough")
        depth_array = np.array(cv2_depth_img, dtype=np.float32)
        print depth_array
        #print depth_array[36,532]
        cv2.imwrite('../../../../tensorflow_proj/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.jpeg', cv2_depth_img)
        np.save('../../../../tensorflow_proj/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.npy', depth_array)
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        stone_img_count = stone_img_count + 1
        is_send_stone_img = 0
        '''
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        #cv2_img = bridge.imgmsg_to_cv2(msg, "bgr8")
        #cv2.imwrite('/home/yuhin/ws_moveit/src/pickpack/Mask_RCNN/log/'+str(carton_img_count)+'.jpeg', cv2_img)
        rospy.sleep(1)
        #print('from image callback, publish /carton_img_index')
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        #stone_img_count = stone_img_count + 1
        is_send_stone_img = 0
        '''

In [ ]:
#image_topic = '/camera/color/image_raw' #"/usb_cam/image_raw"

# Set up your subscriber and define its callback
#rospy.Subscriber(image_topic, Image, image_callback)
image_color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
image_aligned_depth_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
#info_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/camera_info', CameraInfo)

ts = message_filters.TimeSynchronizer([image_color_sub, image_aligned_depth_sub], 10)
ts.registerCallback(image_callback)

In [ ]:
#stone_img_count = 0

try:
    Robotiq.goto(robotiq_client, pos=0.022, speed=0.5, force=10, block=False)
    go_to_home(0.1, 0.3)
    is_send_stone_img = 1
    
except rospy.ROSInterruptException: pass


In [ ]:
execute()